In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import os

In [ ]:
#network and training parameters:
EPOCHS = 100 #how many iterations we run the training set
BATCH_SIZE = 128
VERBOSE = 1 #no idea what this is but lets see...
NB_CLASSES = 10 #10 digits
N_HIDDEN = 1000 #number of nodes in hidden layer
VAL_SPLIT = 0.2  #how much of sample is reserved for validation


In [ ]:

X_train=np.loadtxt('training_data/Xtrain_av_clean.csv',delimiter=',')
Y_train=np.loadtxt('training_data/Ytrain_av_clean.csv',delimiter=',')
X_trainloc=np.loadtxt('training_data/Xtrainloc_av_clean.csv',delimiter=',')
#X_train=preprocessing.normalize(X_train)
#X_train=X_train[:,150:350]
#scalar=preprocessing.StandardScaler()
#scalar.fit(X_train)
#X_train=scalar.transform(X_train)
#X_train=X_train

Xtrain, Xtest, Ytrain, Ytest, Xtrainl, Xtestl = train_test_split(X_train, Y_train, X_trainloc, test_size=0.2)
print(Xtrain.shape,Ytrain.shape,Xtest.shape,Ytest.shape)

In [ ]:
#got the training set ... now write the neural net
#buid the model
#SGD stochastic gradient descent... 
IMGG=2000
IMGL=200

#add hidden layers
ip = keras.layers.Input(shape=(IMGG,),name='Input')
ipl = keras.layers.Input(shape=(IMGL,),name='Inputl')
x = keras.layers.BatchNormalization()(ip)
x = keras.layers.Dense(512,name='dense_layer',activation='tanh')(x)
x = keras.layers.Dense(512,name='dense_layer_2',activation='tanh')(x)
x = keras.layers.Dense(128,name='dense_layer_2b',activation='tanh')(x)
x = keras.layers.Dense(128,name='dense_layer_2c',activation='relu')(x)
x = keras.layers.Dense(64,name='dense_layer_3',activation='relu')(x)

xl = keras.layers.BatchNormalization()(ipl)
xl = keras.layers.Dense(128,name='Ldense_layer',activation='tanh')(xl)
xl = keras.layers.Dense(128,name='Ldense_layer_2',activation='tanh')(xl)
xl = keras.layers.Dense(64,name='Ldense_layer_2b',activation='tanh')(xl)
xl = keras.layers.Dense(64,name='Ldense_layer_2c',activation='relu')(xl)
xl = keras.layers.Dense(32,name='Ldense_layer_3',activation='relu')(xl)

x2 = keras.layers.Concatenate()([xl,x])
x2 = keras.layers.Dense(16,name='classify1',activation='relu')(x2)
x2 = keras.layers.Dense(1,name='classify2',activation='relu')(x2)
x2 = keras.layers.Dense(2,name='classify3',activation='softmax')(x2)
DenseNN = keras.Model(inputs=[ip,ipl], outputs=x2,name='Convolutional_NN')
DenseNN.summary()
DenseNN.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00005), loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history=DenseNN.fit([np.asarray(Xtrain),np.asarray(Xtrainl)],np.asarray(Ytrain), batch_size=64, epochs=100,
     verbose=1, validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#plt.savefig('present_fullyconnected_loss_glob.png')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('present_fullyconnected_acc_global.png')
plt.show()

In [ ]:
test_loss, test_acc = DenseNN.evaluate(np.array(Xtest), np.array(Ytest))
print('Test accuracy:', test_acc)
Ypred_raw=DenseNN.predict(np.array(Xtest))
Ypred=np.argmax(Ypred_raw, axis=1)
Ytest_new=np.argmax(Ytest,axis=1)
cm = confusion_matrix(Ytest_new, Ypred)
print(cm)